In [1]:
from pyspark.sql import functions as F

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
5,application_1637290684416_0006,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
import re
import numpy as np

# load up all of the documents in the corpus
corpus = sc.textFile(
    "s3://chrisjermainebucket/comp330_A5/TestingDataOneLinePerDoc.txt")

# each entry in validLines will be a line from the text file (that has an id)
validLines = corpus.filter(lambda x: 'id' in x)

# now we transform it into a bunch of (docID, text) pairs
keyAndText = validLines.map(
    lambda x: (x[x.index('id="') + 4:x.index('" url=')], x[x.index('">') + 2:]))

# now we split the text in each (docID, text) pair into a list of words
# after this, we have a data set with (docID, ["word1", "word2", "word3", ...])
# we have a bit of fancy regular expression stuff here to make sure that we do not
# die on some of the documents
regex = re.compile('[^a-zA-Z]')
keyAndListOfWords = keyAndText.map(
    lambda x: (str(x[0]), regex.sub(' ', x[1]).lower().split()))

# now get the top 20,000 words... first change (docID, ["word1", "word2", "word3", ...])
# to ("word1", 1) ("word2", 1)...
allWords = keyAndListOfWords.flatMap(lambda x: ((j, 1) for j in x[1]))

# now, count all of the words, giving us ("word1", 1433), ("word2", 3423423), etc.
allCounts = allWords.reduceByKey(lambda a, b: a + b)

# and get the top 20,000 words in a local array
# each entry is a ("word1", count) pair
topWords = allCounts.top(20000, lambda x: x[1])

# And we'll create a RDD that has a bunch of (word, dictNum) pairs
# start by creating an RDD that has the number 0 thru 20000
# 20000 is the number of words that will be in our dictionary
twentyK = sc.parallelize(range(20000))

# now, we transform (0), (1), (2), !!... to ("mostcommonword", 1) ("nextmostcommon
# the number will be the spot in the dictionary used to tell us where the word i
# HINT: make use of topWords in the lambda that you supply
dictionary = twentyK.map(lambda x: (topWords[x][0], x))

# finally, print out some of the dictionary, just for debugging
topWordsDict = dictionary.collect()  # print(dictionary)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
topWordsDict = dict(topWordsDict)
topWordsDict['applicant']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

604

In [4]:
topWordsDict['and']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2

In [5]:
topWordsDict['attack']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

515

In [6]:
topWordsDict['protein']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3681

In [7]:
topWordsDict['car']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

635

## Task 2: Training
- Create TFIDF vectors as feature vectors
- Create label if document has 'AU' in docID

In [8]:
import re
import numpy as np

# load up all of the documents in the corpus
corpus = sc.textFile(
    "s3://chrisjermainebucket/comp330_A5/TestingDataOneLinePerDoc.txt")

# each entry in validLines will be a line from the text file (that has an id)
validLines = corpus.filter(lambda x: 'id' in x)

# now we transform it into a bunch of (docID, text) pairs
keyAndText = validLines.map(
    lambda x: (x[x.index('id="') + 4:x.index('" url=')], x[x.index('">') + 2:]))

# now we split the text in each (docID, text) pair into a list of words
# after this, we have a data set with (docID, ["word1", "word2", "word3", ...])
# we have a bit of fancy regular expression stuff here to make sure that we do not
# die on some of the documents
regex = re.compile('[^a-zA-Z]')
keyAndListOfWords = keyAndText.map(
    lambda x: (str(x[0]), regex.sub(' ', x[1]).lower().split()))

# now get the top 20,000 words... first change (docID, ["word1", "word2", "word3", ...])
# to ("word1", 1) ("word2", 1)...
word_docID_unmapped = keyAndListOfWords.flatMap(lambda x:
                                                ((j, x[0]) for j in x[1]))
word_docID_unmapped_local = word_docID_unmapped.collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
word_docID_unmapped = word_docID_unmapped.join(dictionary)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
word_docID_mapped = word_docID_unmapped.map(lambda x: (x[1][0], x[1][1]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
word_docID_mapped = word_docID_mapped.groupByKey()
word_docID_mapped = word_docID_mapped.map(lambda x: (x[0], list(x[1])))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
import numpy as np

def list_to_np(ls):
    arr = np.zeros(20000)
    for i in ls:
        arr[i] += 1
    return arr

res = word_docID_mapped.map(lambda x: ((x[0]), list_to_np(x[1])))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
# TF(i, d) = (Number of occurences of word i in d) / (Total number of words in d)
# IDF(i) = log((Size of corpus (number of docs)) / (Number of documents having word i))
import numpy as np

corpus_size = res.count()


def calc_IDF(whole_arr):
    temp_arr = whole_arr.map(lambda x: np.clip(x[1], 0, 1))
    temp_arr = temp_arr.sum()
    temp_arr = corpus_size * np.reciprocal(temp_arr)
    temp_arr = np.log(temp_arr)
    return temp_arr


IDF = calc_IDF(res)


def calc_TFIDF(arr):
    t = np.sum(arr)
    TF = arr / t
    TFIDF = np.multiply(TF, IDF)
    return TFIDF


res_TFIDF = res.map(lambda x: (x[0], calc_TFIDF(x[1])))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [67]:
# Normalize the TFIDF vector
doc_cnt = res_TFIDF.count()
y_train = res_TFIDF.map(lambda x: 1 if 'AU' in x[0] else 0)
x_train = res_TFIDF.map(lambda x: x[1])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [68]:
## TODO: Normalize TFIDF without collecting
from operator import add
mean_vec = res_TFIDF.map(lambda x: ("key", x[1]))
mean_vec = mean_vec.reduceByKey(add)
mean_vec = mean_vec.map(lambda x: x[1])
mean_vec_local = mean_vec.collect()[0]/doc_cnt # Get mean_vec this way is better


std_vec = res_TFIDF.map(lambda x: ("key", 10000 * np.square(x[1] - mean_vec_local)))
std_vec = std_vec.reduceByKey(add)
std_vec = std_vec.map(lambda x:x[1])
std_vec_local = np.sqrt(std_vec.collect()[0] / doc_cnt)
std_vec_local[std_vec_local == 0] = 1
train_data = res_TFIDF.map(lambda x: (1 if 'AU' in x[0] else 0,
                                      (x[1] - mean_vec_local) / std_vec_local))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [69]:
# evaluates the loss function and returns the loss
#
# x is the data set
# y is the labels
# w is the current set of weights
# c is the weight of the slack variables
from operator import add

def f(train_y_x, w, c):
    l2 = np.linalg.norm(w, 2)
    loss_per_row = train_y_x.map(lambda p: ("key", (-1 * p[0] * np.dot(p[1], w)) + np.log(1 + np.exp(np.dot(p[1], w))) + c * l2 * l2))
    loss = loss_per_row.reduceByKey(add).collect()
    loss = loss[0][1]
    print(f"--loss is {loss}")
    return loss

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [70]:
# evaluates and returns the gradient
#
# x is the data set
# y is the labels
# w is the current set of weights
# c is the weight of the slack variables
#
def gradient(train_y_x, w, c):
    l2 = np.linalg.norm(w, 2)
    grad_per_row = train_y_x.map(lambda p: -1 * p[0] * p[1] + p[1] * (np.exp(np.dot(p[1], w)) / (1 + np.exp(np.dot(p[1], w)))) + c * w)
    accumulator = np.zeros(20000)
    grad = grad_per_row.fold(accumulator, lambda a, b: a + b)
    print(f"--grad is {grad}")
    return grad

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [71]:
# make predictions using all of the data points in x
# print ‘success’ or ‘failure’ depending on whether the
# prediction is correct
#
# x is the data set
# y is the labels
# w is the current set of weights
#

def predict(data, label, w):
    correct = 0
    predictTrueCnt = 0  # num we say are true
    trueCnt = 0  # num that are true
    absTrueCnt = 0  # num we say true that are actually true
    
    pred = data.map(lambda x: np.dot(x, w))
    pred = pred.collect()
    y = label.collect()
    
    for index in range(len(y)):
        if (pred[index] > 1) and (y[index] > 0):
            print('success')
            predictTrueCnt += 1
            trueCnt += 1
            correct += 1
            absTrueCnt += 1
        elif (pred[index] < 0.5 and (y[index] == 0)):
            print('success')
            correct += 1
        else:
            if (pred[index] > -0.5):
                predictTrueCnt += 1
            if (y[index] > 0):
                trueCnt += 1
            print('failure')
    print(pred)
    print(absTrueCnt, predictTrueCnt, trueCnt)
    precision = absTrueCnt / predictTrueCnt
    recall = absTrueCnt / trueCnt
    print('%d out of %d correct.' % (correct, len(y)))
    print(f"Precision is: {precision}")
    print(f"Recall is: {recall}")
    print(f"F1 score is: {2*precision*recall/(precision+recall)}")
    
    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [72]:
# performs gradient descent optimization, returns the learned set of weights
# uses the bold driver to set the learning rate
#
# x is the data set
# y is the labels
# w is the current set of weights  to start with
# c is the weight of the slack variable
#
def gd_optimize (train_x_y, w, c):
    rate = 0.1
    w_last = w + np.full(20000, 1.0) 
    while abs(f(train_x_y, w, c) - f(train_x_y, w_last, c)) > 0.0001:
        w_last = w 
        w = w - rate * gradient(train_x_y, w, c)
        if f(train_x_y, w, c) > f(train_x_y, w_last, c):
            rate = rate * .5
        else:
            rate = rate * 1.1
        print (f(train_x_y, w, c))
    return w

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [73]:
## Runner
# import numpy.matlib
# w = np.random.rand(20000)
# generate random Gaussian values
from random import seed
from random import gauss

w= np.random.normal(0, 0.1, size=(20000))
w = gd_optimize(train_data, w, .01)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

--loss is 50170.236053751876
--loss is 3805388.3715752973
--grad is [ 17.44640434 -16.74183397 -34.82721146 ... -17.58105328 -13.19527687
  25.68300875]
--loss is 11785645.767698023
--loss is 50170.23605375186
--loss is 11785645.767698018
11785645.767698018
--loss is 11785645.76769802
--loss is 50170.23605375186
--grad is [-310.78862709  297.45935859  615.15790845 ...  312.55964781  235.50090728
 -455.92031603]
--loss is 824914102.1086795
--loss is 11785645.767698016
--loss is 824914102.1086798
824914102.1086798
--loss is 824914102.1086798
--loss is 11785645.767698018
--grad is [ 2602.70349016 -2488.87414419 -5139.36001673 ... -2615.67988594
 -1972.27738903  3814.03439521]
--loss is 11181524631.363514
--loss is 824914102.1086797
--loss is 11181524631.36351
11181524631.36351
--loss is 11181524631.363508
--loss is 824914102.1086798
--grad is [ -9585.61401408   9162.94074787  18913.29235509 ...   9630.59955607
   7263.06953201 -14041.29098332]
--loss is 20095747972.97563
--loss is 1118152

In [74]:
ind = np.argpartition(w, -50)[-50:]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [75]:
for i in ind:
    for word, pos in topWordsDict.items():
        if pos == i:
            print(word)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

matters
magistrate
court
consider
jurisdictional
amp
act
regard
respondents
appellant
judgment
decision
discretion
notice
orders
sought
ltd
matter
that
proceedings
any
not
relevantly
respondent
clr
mr
pursuant
hearing
submissions
tribunal
whether
hca
fcr
gummow
fcafc
pty
relation
circumstances
applicant
proceeding
fca
application
relevant
affidavit
reasons
claim
appeal
alr
evidence
satisfied

In [76]:
# predict(x_train, y_train, w)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Task 3: Evaluating 

#### Transform raw testing documents into valid TFIDF test data & label

In [77]:
res_TFIDF = res_TFIDF.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [78]:
import re
import numpy as np

# load up all of the documents in the corpus
test_corpus = sc.textFile(
    "s3://chrisjermainebucket/comp330_A5/SmallTrainingDataOneLinePerDoc.txt")

# each entry in validLines will be a line from the text file (that has an id)
test_validLines = test_corpus.filter(lambda x: 'id' in x)

# now we transform it into a bunch of (docID, text) pairs
test_keyAndText = test_validLines.map(lambda x: (x[x.index('id="') + 4:x.index('" url=')], x[x.index('">') + 2:]))

# now we split the text in each (docID, text) pair into a list of words
# after this, we have a data set with (docID, ["word1", "word2", "word3", ...])
# we have a bit of fancy regular expression stuff here to make sure that we do not
# die on some of the documents
regex = re.compile('[^a-zA-Z]')
test_keyAndListOfWords = test_keyAndText.map(lambda x: (str(x[0]), regex.sub(' ', x[1]).lower().split()))

# now get the top 20,000 words... first change (docID, ["word1", "word2", "word3", ...])
# to ("word1", 1) ("word2", 1)...
test_word_docID_unmapped = test_keyAndListOfWords.flatMap(lambda x:((j, x[0]) for j in x[1]))
test_word_docID_unmapped_local = test_word_docID_unmapped.collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [79]:
test_word_docID_unmapped_local = test_word_docID_unmapped.join(dictionary)
test_word_docID_mapped = test_word_docID_unmapped_local.map(lambda x: (x[1][0], x[1][1]))
test_word_docID_mapped = test_word_docID_mapped.groupByKey()
test_word_docID_mapped = test_word_docID_mapped.map(lambda x: (x[0], list(x[1])))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [80]:
import numpy as np

def list_to_np(ls):
    arr = np.zeros(20000)
    for i in ls:
        arr[i] += 1
    return arr

test_res = test_word_docID_mapped.map(lambda x: ((x[0]), list_to_np(x[1])))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [81]:
# TF(i, d) = (Number of occurences of word i in d) / (Total number of words in d)
# IDF(i) = log((Size of corpus (number of docs)) / (Number of documents having word i))
# import numpy as np

test_corpus_size = test_res.count()


def calc_test_IDF(whole_arr):
    temp_arr = whole_arr.map(lambda x: np.clip(x[1], 0, 1))
    temp_arr = temp_arr.sum()
    temp_arr = test_corpus_size * np.reciprocal(temp_arr)
    temp_arr = np.log(temp_arr)
    return temp_arr

IDF = calc_test_IDF(test_res)

def calc_test_TFIDF(arr):
    t = np.sum(arr)
    TF = arr / t
    TFIDF = np.multiply(TF, IDF)
    return TFIDF

test_res_TFIDF = test_res.map(lambda x: (x[0], calc_test_TFIDF(x[1])))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [82]:
test_doc_cnt = test_res_TFIDF.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [83]:
test_mean_vec = test_res_TFIDF.map(lambda x: ("key", x[1]))
test_mean_vec = test_mean_vec.reduceByKey(add)
test_mean_vec = test_mean_vec.map(lambda x: x[1])
test_mean_vec_local = test_mean_vec.collect()[0] / test_doc_cnt # Get mean_vec this way is better

test_std_vec = test_res_TFIDF.map(lambda x: ("key", np.square(x[1] - test_mean_vec_local)))
test_std_vec = test_std_vec.reduceByKey(add)
test_std_vec = test_std_vec.map(lambda x:x[1])
test_std_vec_local = np.sqrt(test_std_vec.collect()[0] / test_doc_cnt)
test_std_vec_local[test_std_vec_local == 0] = 1
test_data = test_res_TFIDF.map(lambda x: (1 if 'AU' in x[0] else 0,
                                      (x[1] - test_mean_vec_local) / test_std_vec_local))

x_test = test_data.map(lambda x: x[1]).collect()
y_test = test_data.map(lambda x: x[0]).collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [93]:
def predict_test(x, y, w):
     correct = 0  
     predictTrueCnt = 0  # num we say are true
     trueCnt = 0  # num that are true
     absTrueCnt = 0 # num we say true that are actually true
     for index in range(len(y)):
          print(np.dot(x[index], w), x[index].shape, w.shape)
          if ((np.dot(x[index], w) > -2) and (y[index] > 0)):
               print('success')
               predictTrueCnt += 1
               trueCnt += 1
               correct += 1
               absTrueCnt += 1
          elif ((np.dot(x[index], w) < 5) and (y[index] == 0)):
               print ('success')
               correct += 1
          else:
               if (np.dot(x[index], w) > -1):
                    print(f"--FP index = {index}")
                    predictTrueCnt += 1
               if (y[index] > 0):
                    trueCnt += 1
               print ('failure')
     precision = absTrueCnt / predictTrueCnt
     recall = absTrueCnt / trueCnt
     print ('%d out of %d correct.' % (correct, len(y))) 
     print (f"Precision is: {precision}")
     print (f"Recall is: {recall}")
     print (f"F1 score is: {2 * precision * recall / (precision+recall)}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [85]:
x_test = np.nan_to_num(x_test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [94]:
predict_test(x_test, y_test, w)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-1.6080044949110894 (20000,) (20000,)
success
1.3307667676298127 (20000,) (20000,)
success
1.5970477785148882 (20000,) (20000,)
success
-0.21194574702628907 (20000,) (20000,)
success
-0.31559334174258186 (20000,) (20000,)
success
-2.0468650407371336 (20000,) (20000,)
success
-1.4093722534005333 (20000,) (20000,)
success
-1.3713977597525289 (20000,) (20000,)
success
-2.412661212835896 (20000,) (20000,)
success
-1.1798682342595708 (20000,) (20000,)
success
-1.8867219337131327 (20000,) (20000,)
success
-1.5401343613888536 (20000,) (20000,)
success
0.40904164180077274 (20000,) (20000,)
success
-0.4415749218944958 (20000,) (20000,)
success
-1.5242206765728115 (20000,) (20000,)
success
1.582806392573141 (20000,) (20000,)
success
8.055240316531318 (20000,) (20000,)
--FP index = 16
failure
-2.124184214173573 (20000,) (20000,)
success
0.9625073562531429 (20000,) (20000,)
success
0.08041646160567616 (20000,) (20000,)
success
-1.2503396707321741 (20000,) (20000,)
success
1.02728269404985 (20000,)